In [2]:
import pandas as pd
import numpy as np
import os

In [71]:
def binarySearchAsc(mItem, mList):

    startIndex = 0
    endIndex = len(mList) - 1
    
    while(startIndex <= endIndex):
        target = int((startIndex + endIndex) / 2)
        
        if(mItem == mList[target]):
            return [True, target]
        elif(mItem < mList[target]):
            endIndex = target - 1
        else:
            startIndex = target + 1
            
    if(endIndex == -1):
        return [False, startIndex]
    if(startIndex > endIndex):
        return [False, startIndex]
    return [False, target]

In [144]:
df = pd.read_csv('data/champions0.csv', index_col=False, usecols=[1,2,3,4,5])
for i in range(8):
    df = pd.concat([df, pd.read_csv('data/champions{}.csv'.format(i * 20000), index_col=False, usecols=[1,2,3,4,5])])
df = df.drop_duplicates()

In [145]:
trollerList = df[(df['kda'] < 0.15) & (df['kda'] > 0)]['matchID']

In [146]:
trollerList = trollerList.sort_values()

In [147]:
trollerList = trollerList.reset_index(drop=True)

In [148]:
trollerList = trollerList.unique()

In [149]:
def trollerCheck(x):
    result = binarySearchAsc(x[0], trollerList)
    
    if result[0] == True:
        pass
    else:
        return x

finalList = df.apply(trollerCheck, axis=1)

In [150]:
finalList = finalList[finalList['matchID'].isna() == False]

In [151]:
finalList[finalList.columns[:-1]].to_csv('data/champions_final.csv', index=False)

In [152]:
matchdf = pd.read_csv('data/champions_final.csv')
matchdf

,matchID,championID,position,win
0,KR_6007289435,268.0,TOP,False
1,KR_6007289435,254.0,JUNGLE,False
2,KR_6007289435,517.0,MIDDLE,False
3,KR_6007289435,221.0,BOTTOM,False
4,KR_6007289435,497.0,UTILITY,False
...,...,...,...,...
437575,KR_5980422724,166.0,TOP,False
437576,KR_5980422724,76.0,JUNGLE,False
437577,KR_5980422724,13.0,MIDDLE,False
437578,KR_5980422724,81.0,BOTTOM,False


In [153]:
groupMatch = matchdf.groupby(['matchID', 'win'])

In [168]:
t = pd.DataFrame(groupMatch)

In [197]:
def getChamInfo(x):
    return x[['championID', 'position']]
    
chamInfo = t[1].apply(getChamInfo)

In [211]:
resultInfo = t[0].apply(lambda x: x[1])

In [354]:
x_data = chamInfo.apply(lambda x : x.to_numpy().transpose()).to_numpy()

In [356]:
y_data = resultInfo.to_numpy()

In [357]:
x_data[0], y_data[0]

(array([[17.0, 75.0, 104.0, 81.0, 101.0],
        ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']], dtype=object),
 False)

In [366]:
x_data = x_data.reshape(-1, 1)

In [378]:
x_position = np.apply_along_axis(lambda x : x[0][1], 1, x_data)

In [379]:
x_champion = np.apply_along_axis(lambda x : x[0][0], 1, x_data)

In [61]:
pd.DataFrame(x_position).to_csv('data/x_position.csv', index=False)

In [389]:
pd.DataFrame(x_champion).to_csv('data/x_champion.csv', index=False)

In [390]:
pd.DataFrame(y_data).to_csv('data/y_data.csv', index=False)

In [31]:
x_position = pd.read_csv('data/x_position.csv').to_numpy()
x_champion = pd.read_csv('data/x_champion.csv').to_numpy()
y_data = pd.read_csv('data/y_data.csv').to_numpy().reshape(-1)

In [58]:
nanList = []
for i, position in enumerate(x_position):
    for x in position:
        if(x not in ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']):
            nanList.append(i)
            break

In [59]:
nanList

[]

In [55]:
nanConvList =  x_position[nanList]
for position in nanConvList:
    for i in range(5):
        if(position[i].__class__ != 'TOP'.__class__):
            if(i == 0):
                position[i] = 'TOP'
            if(i == 1):
                position[i] = 'JUNGLE'
            if(i == 2):
                position[i] = 'MIDDLE'
            if(i == 3):
                position[i] = 'BOTTOM'
            if(i == 4):
                position[i] = 'UTILITY'

In [57]:
for i, i0 in enumerate(nanList):
    x_position[i0] = nanConvList[i]